In [1]:
import torch
import pandas as pd
import numpy as np
import torch.nn.functional as F

In [2]:
data = pd.read_csv('../dataset/criteo-uplift-v2.1/criteo-uplift-v2.1.csv')

In [3]:
data.shape

(13979592, 16)

In [6]:
demo_data = data.sample(10000)

In [7]:
demo_data.shape

(10000, 16)

In [8]:
demo_data.head()

,f0,f1,f2,f3,f4,f5,f6,f7,f8,f9,f10,f11,treatment,conversion,visit,exposure
11331931,12.616365,10.059654,8.246414,4.679882,10.280525,4.115453,0.294443,4.833815,3.835145,27.760385,5.300375,-0.168679,1,0,0,0
4822801,22.106220,10.059654,8.214383,4.679882,10.280525,3.013064,-5.987667,9.646543,3.971858,13.190056,5.300375,-0.168679,1,0,0,0
9723940,26.136972,10.059654,9.017182,4.679882,10.280525,4.115453,-3.993764,4.833815,3.934656,13.190056,5.300375,-0.168679,1,0,0,0
10161609,12.616365,10.059654,8.910057,4.679882,10.280525,4.115453,0.294443,4.833815,3.943716,13.190056,5.300375,-0.168679,1,0,0,0
12651160,20.816374,10.059654,8.214383,4.679882,10.280525,4.115453,-4.595460,4.833815,3.971858,13.190056,5.300375,-0.168679,1,0,0,0


## treatment analysis

In [11]:
data_t1 = demo_data[demo_data['treatment']==1]
data_t0 = demo_data[demo_data['treatment']==0]

In [12]:
data_t1.describe()

,f0,f1,f2,f3,f4,f5,f6,f7,f8,f9,f10,f11,treatment,conversion,visit,exposure
count,8517.000000,8517.000000,8517.000000,8517.000000,8517.000000,8517.000000,8517.000000,8517.000000,8517.000000,8517.000000,8517.000000,8517.000000,8517.0,8517.000000,8517.00000,8517.000000
mean,19.564502,10.069381,8.445364,4.150432,10.337339,4.033402,-4.146832,5.081416,3.933280,16.140123,5.331447,-0.171012,1.0,0.003288,0.05037,0.033932
std,5.360378,0.097192,0.298242,1.376350,0.351959,0.413052,4.572371,1.152822,0.057497,7.151473,0.163915,0.023271,0.0,0.057246,0.21872,0.181065
min,12.616365,10.059654,8.214383,-5.015458,10.280525,-3.155091,-25.220187,4.833815,3.660531,13.190056,5.300375,-0.674717,1.0,0.000000,0.00000,0.000000
25%,12.616365,10.059654,8.214383,4.679882,10.280525,4.115453,-6.699321,4.833815,3.910792,13.190056,5.300375,-0.168679,1.0,0.000000,0.00000,0.000000
50%,21.873661,10.059654,8.214383,4.679882,10.280525,4.115453,-2.411115,4.833815,3.971858,13.190056,5.300375,-0.168679,1.0,0.000000,0.00000,0.000000
75%,24.350303,10.059654,8.712828,4.679882,10.280525,4.115453,0.294443,4.833815,3.971858,13.190056,5.300375,-0.168679,1.0,0.000000,0.00000,0.000000
max,26.744923,11.974826,9.051847,4.679882,16.822120,4.115453,0.294443,11.994548,3.971858,61.975606,6.473484,-0.168679,1.0,1.000000,1.00000,1.000000


In [13]:
data_t0.describe()

,f0,f1,f2,f3,f4,f5,f6,f7,f8,f9,f10,f11,treatment,conversion,visit,exposure
count,1483.000000,1483.000000,1483.000000,1483.000000,1483.000000,1483.000000,1483.000000,1483.000000,1483.000000,1483.000000,1483.000000,1483.000000,1483.0,1483.000000,1483.000000,1483.0
mean,19.547845,10.069238,8.449070,4.262641,10.327573,4.034709,-4.023343,5.068701,3.933380,16.118875,5.329268,-0.170535,0.0,0.002023,0.043830,0.0
std,5.374364,0.093724,0.300234,1.182484,0.280533,0.434626,4.446829,1.127907,0.056009,7.116966,0.155738,0.019081,0.0,0.044947,0.204786,0.0
min,12.616365,10.059654,8.214383,-2.917251,10.280525,-3.422733,-21.249629,4.833815,3.715747,13.190056,5.300375,-0.506038,0.0,0.000000,0.000000,0.0
25%,12.616365,10.059654,8.214383,4.679882,10.280525,4.115453,-6.359690,4.833815,3.910792,13.190056,5.300375,-0.168679,0.0,0.000000,0.000000,0.0
50%,21.880676,10.059654,8.214383,4.679882,10.280525,4.115453,-2.411115,4.833815,3.971858,13.190056,5.300375,-0.168679,0.0,0.000000,0.000000,0.0
75%,24.357839,10.059654,8.743680,4.679882,10.280525,4.115453,0.294443,4.833815,3.971858,13.190056,5.300375,-0.168679,0.0,0.000000,0.000000,0.0
max,26.736381,11.974826,9.051743,4.679882,13.875414,4.115453,0.294443,11.890481,3.971858,55.848000,6.455756,-0.168679,0.0,1.000000,1.000000,0.0


## exposure analysis

In [14]:
data_t1 = demo_data[demo_data['exposure']==1]
data_t0 = demo_data[demo_data['exposure']==0]

In [15]:
data_t1.describe()

,f0,f1,f2,f3,f4,f5,f6,f7,f8,f9,f10,f11,treatment,conversion,visit,exposure
count,289.000000,289.000000,289.000000,289.000000,289.000000,289.000000,289.000000,289.000000,289.000000,289.000000,289.000000,289.000000,289.0,289.000000,289.000000,289.0
mean,15.855332,10.178104,8.420144,1.170409,10.675920,3.544014,-11.318546,6.050128,3.867454,25.048976,5.414594,-0.188592,1.0,0.058824,0.418685,1.0
std,3.335576,0.346370,0.230437,2.511381,1.024494,1.073252,5.704562,2.144545,0.079645,12.314765,0.274990,0.068894,0.0,0.235702,0.494199,0.0
min,12.616365,10.059654,8.214383,-5.015458,10.280525,-3.155091,-24.142274,4.833815,3.675856,13.190056,5.300375,-0.659011,1.0,0.000000,0.000000,1.0
25%,13.566675,10.059654,8.219062,-0.977357,10.280525,3.013064,-15.275735,4.833815,3.804955,13.190056,5.300375,-0.168679,1.0,0.000000,0.000000,1.0
50%,14.441813,10.059654,8.328704,1.267425,10.280525,4.115453,-12.110436,4.833815,3.863287,21.416100,5.300375,-0.168679,1.0,0.000000,0.000000,1.0
75%,17.215633,10.059654,8.549724,3.359763,10.280525,4.115453,-7.011752,6.552440,3.955396,35.621104,5.300375,-0.168679,1.0,0.000000,1.000000,1.0
max,26.720357,11.974826,9.048930,4.679882,15.814863,4.115453,0.294443,11.991396,3.971858,56.901043,6.443264,-0.168679,1.0,1.000000,1.000000,1.0


In [16]:
data_t0.describe()

,f0,f1,f2,f3,f4,f5,f6,f7,f8,f9,f10,f11,treatment,conversion,visit,exposure
count,9711.000000,9711.000000,9711.000000,9711.000000,9711.000000,9711.000000,9711.000000,9711.000000,9711.000000,9711.000000,9711.000000,9711.000000,9711.000000,9711.000000,9711.000000,9711.0
mean,19.672344,10.066124,8.446680,4.256253,10.325771,4.048166,-3.914543,5.050645,3.935254,15.871750,5.328640,-0.170416,0.847287,0.001442,0.038410,0.0
std,5.371940,0.075529,0.300291,1.189182,0.293244,0.370071,4.336611,1.092924,0.055292,6.756707,0.157513,0.019499,0.359730,0.037944,0.192194,0.0
min,12.616365,10.059654,8.214383,-4.485059,10.280525,-3.422733,-25.220187,4.833815,3.660531,13.190056,5.300375,-0.674717,0.000000,0.000000,0.000000,0.0
25%,12.616365,10.059654,8.214383,4.679882,10.280525,4.115453,-6.359690,4.833815,3.915574,13.190056,5.300375,-0.168679,1.000000,0.000000,0.000000,0.0
50%,21.984751,10.059654,8.214383,4.679882,10.280525,4.115453,-2.411115,4.833815,3.971858,13.190056,5.300375,-0.168679,1.000000,0.000000,0.000000,0.0
75%,24.384927,10.059654,8.729383,4.679882,10.280525,4.115453,0.294443,4.833815,3.971858,13.190056,5.300375,-0.168679,1.000000,0.000000,0.000000,0.0
max,26.744923,11.974826,9.051847,4.679882,16.822120,4.115453,0.294443,11.994548,3.971858,61.975606,6.473484,-0.168679,1.000000,1.000000,1.000000,0.0


## visit analysis

In [17]:
data_t1 = demo_data[demo_data['visit']==1]
data_t0 = demo_data[demo_data['visit']==0]

In [18]:
data_t1.describe()

,f0,f1,f2,f3,f4,f5,f6,f7,f8,f9,f10,f11,treatment,conversion,visit,exposure
count,494.000000,494.000000,494.000000,494.000000,494.000000,494.000000,494.000000,494.000000,494.000000,494.000000,494.000000,494.000000,494.000000,494.000000,494.0,494.000000
mean,16.325509,10.116138,8.398167,3.026910,10.772317,3.770880,-6.900183,5.512875,3.811533,32.315115,5.495390,-0.195841,0.868421,0.062753,1.0,0.244939
std,4.236350,0.245470,0.183278,2.294567,0.988215,0.966104,6.611788,1.761624,0.061492,11.737811,0.344033,0.077500,0.338375,0.242764,0.0,0.430487
min,12.616365,10.059654,8.214383,-5.015458,10.280525,-3.422733,-24.142274,4.833815,3.660531,13.190056,5.300375,-0.674717,0.000000,0.000000,1.0,0.000000
25%,12.616365,10.059654,8.258496,1.614662,10.280525,4.115453,-12.110436,4.833815,3.767257,23.570168,5.300375,-0.168679,1.000000,0.000000,1.0,0.000000
50%,14.262200,10.059654,8.334011,4.679882,10.280525,4.115453,-5.987667,4.833815,3.806837,33.081817,5.300375,-0.168679,1.000000,0.000000,1.0,0.000000
75%,19.621415,10.059654,8.490385,4.679882,11.029584,4.115453,0.294443,4.833815,3.852313,41.330780,5.583779,-0.168679,1.000000,0.000000,1.0,0.000000
max,26.720357,11.974826,9.031214,4.679882,15.814863,4.115453,0.294443,11.991396,3.971858,61.975606,6.459516,-0.168679,1.000000,1.000000,1.0,1.000000


In [19]:
data_t0.describe()

,f0,f1,f2,f3,f4,f5,f6,f7,f8,f9,f10,f11,treatment,conversion,visit,exposure
count,9506.000000,9506.000000,9506.000000,9506.000000,9506.000000,9506.000000,9506.000000,9506.000000,9506.000000,9506.000000,9506.000000,9506.000000,9506.000000,9506.0,9506.0,9506.000000
mean,19.730225,10.066929,8.448394,4.226324,10.313211,4.047248,-3.984483,5.057010,3.939622,15.296240,5.322588,-0.169647,0.850831,0.0,0.0,0.017673
std,5.361332,0.081167,0.303121,1.254098,0.249406,0.360733,4.373671,1.103568,0.049440,5.680107,0.142269,0.014020,0.356274,0.0,0.0,0.131767
min,12.616365,10.059654,8.214383,-4.485059,10.280525,-2.524863,-25.220187,4.833815,3.680889,13.190056,5.300375,-0.624187,0.000000,0.0,0.0,0.000000
25%,12.616365,10.059654,8.214383,4.679882,10.280525,4.115453,-6.359690,4.833815,3.920995,13.190056,5.300375,-0.168679,1.000000,0.0,0.0,0.000000
50%,22.079882,10.059654,8.214383,4.679882,10.280525,4.115453,-2.411115,4.833815,3.971858,13.190056,5.300375,-0.168679,1.000000,0.0,0.0,0.000000
75%,24.404577,10.059654,8.739433,4.679882,10.280525,4.115453,0.294443,4.833815,3.971858,13.190056,5.300375,-0.168679,1.000000,0.0,0.0,0.000000
max,26.744923,11.974826,9.051847,4.679882,16.822120,4.115453,0.294443,11.994548,3.971858,55.848000,6.473484,-0.168679,1.000000,0.0,0.0,1.000000


In [24]:
ml.links

,item_id,imdb_id,tmdb_id
0,1,114709,862.0
1,2,113497,8844.0
2,3,113228,15602.0
3,4,114885,31357.0
4,5,113041,11862.0
...,...,...,...
9737,193581,5476944,432131.0
9738,193583,5914996,445030.0
9739,193585,6397426,479308.0
9740,193587,8391976,483455.0


In [31]:
ml.ratings

,user_id,item_id,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931
...,...,...,...,...
100831,610,166534,4.0,1493848402
100832,610,168248,5.0,1493850091
100833,610,168250,5.0,1494273047
100834,610,168252,5.0,1493846352
